In [1]:
!nvidia-smi

Fri Sep 24 08:45:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
%%time
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.9.0+cu102.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.9.0+cu102.html
!pip install -q torch-geometric
# !pip install -q torch-scatter
# !pip install -q torch-sparse 

     |████████████████████████████████| 8.0 MB 5.1 MB/s 
     |████████████████████████████████| 2.9 MB 5.0 MB/s 
     |████████████████████████████████| 308 kB 5.4 MB/s 
     |████████████████████████████████| 379 kB 39.9 MB/s 
     |████████████████████████████████| 45 kB 3.5 MB/s 
CPU times: user 169 ms, sys: 45.1 ms, total: 214 ms
Wall time: 15.6 s


In [9]:
!pip show torch

Name: torch
Version: 1.9.0+cu102
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.7/dist-packages
Requires: typing-extensions
Required-by: torchvision, torchtext, fastai


In [10]:
from torch_geometric.nn import HGTConv, Linear

class HGT(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, num_heads, num_layers):
        super().__init__()

        self.lin_dict = torch.nn.ModuleDict()
        for node_type in data.node_types:
            self.lin_dict[node_type] = Linear(-1, hidden_channels)

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HGTConv(hidden_channels, hidden_channels, data.metadata(),
                           num_heads, group='sum')
            self.convs.append(conv)

        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        x_dict = {key: lin(x).relu() for key, lin in self.lin_dict.items()}
        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)
        return self.lin(x_dict['author'])

In [5]:
import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG
from torch_geometric.loader import NeighborLoader

transform = T.ToUndirected()  # Add reverse edge types.
data = OGB_MAG(root='../data', preprocess='metapath2vec', transform=transform)[0]

train_loader = NeighborLoader(
    data,
    # Sample 15 neighbors for each node and each edge type for 2 iterations:
    num_neighbors=[15] * 2,
    # Use a batch size of 128 for sampling training nodes of type "paper":
    batch_size=256,
    input_nodes=('paper', data['paper'].train_mask),
)

batch = next(iter(train_loader))

In [6]:
print(batch)

HeteroData(
  paper={
    x=[38573, 128],
    y=[38573],
    train_mask=[38573],
    val_mask=[38573],
    test_mask=[38573],
    batch_size=256
  },
  author={ x=[8281, 128] },
  institution={ x=[513, 128] },
  field_of_study={ x=[4145, 128] },
  (author, affiliated_with, institution)={ edge_index=[2, 0] },
  (author, writes, paper)={ edge_index=[2, 11210] },
  (paper, cites, paper)={ edge_index=[2, 24359] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 17944] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 1652] },
  (paper, rev_writes, author)={ edge_index=[2, 10579] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 21393] }
)


In [11]:
data

HeteroData(
  paper={
    x=[736389, 128],
    y=[736389],
    train_mask=[736389],
    val_mask=[736389],
    test_mask=[736389]
  },
  author={ x=[1134649, 128] },
  institution={ x=[8740, 128] },
  field_of_study={ x=[59965, 128] },
  (author, affiliated_with, institution)={ edge_index=[2, 1043998] },
  (author, writes, paper)={ edge_index=[2, 7145660] },
  (paper, cites, paper)={ edge_index=[2, 10792672] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 7505078] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 1043998] },
  (paper, rev_writes, author)={ edge_index=[2, 7145660] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 7505078] }
)

In [ ]:
# https://pytorch-geometric.readthedocs.io/en/latest/notes/heterogeneous.html
# https://github.com/pyg-team/pytorch_geometric/blob/master/examples/hetero/to_hetero_mag.py

In [ ]:
import argparse
import os.path as osp
from tqdm import tqdm

import torch
from torch.nn import ReLU
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG
from torch_geometric.loader import NeighborLoader, HGTLoader
from torch_geometric.nn import Sequential, SAGEConv, Linear, to_hetero

# parser = argparse.ArgumentParser()
# parser.add_argument('--use_hgt_loader', action='store_true')
# args = parser.parse_args()

# path = osp.join(osp.dirname(osp.realpath(__file__)), '../data/OGB')
path = '../data'
transform = T.ToUndirected(merge=True)
dataset = OGB_MAG(path, preprocess='metapath2vec', transform=transform)
data = dataset[0]

train_input_nodes = ('paper', data['paper'].train_mask)
val_input_nodes = ('paper', data['paper'].val_mask)
kwargs = {'batch_size': 256, 'num_workers': 2, 'persistent_workers': True}

train_loader = NeighborLoader(data, num_neighbors=[10] * 2, shuffle=True,
                              input_nodes=train_input_nodes, **kwargs)
val_loader = NeighborLoader(data, num_neighbors=[10] * 2,
                            input_nodes=val_input_nodes, **kwargs)
# if not args.use_hgt_loader:
#     train_loader = NeighborLoader(data, num_neighbors=[10] * 2, shuffle=True,
#                                   input_nodes=train_input_nodes, **kwargs)
#     val_loader = NeighborLoader(data, num_neighbors=[10] * 2,
#                                 input_nodes=val_input_nodes, **kwargs)
# else:
#     train_loader = HGTLoader(data, num_samples=[1024] * 4, shuffle=True,
#                              input_nodes=train_input_nodes, **kwargs)
#     val_loader = HGTLoader(data, num_samples=[1024] * 4,
#                            input_nodes=val_input_nodes, **kwargs)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Sequential('x, edge_index', [
    (SAGEConv((-1, -1), 64), 'x, edge_index -> x'),
    ReLU(inplace=True),
    (SAGEConv((-1, -1), 64), 'x, edge_index -> x'),
    ReLU(inplace=True),
    (Linear(-1, dataset.num_classes), 'x -> x'),
])
model = to_hetero(model, data.metadata(), aggr='sum').to(device)


@torch.no_grad()
def init_params():
    # Initialize lazy parameters via forwarding a single batch to the model:
    batch = next(iter(train_loader))
    batch = batch.to(device)
    model(batch.x_dict, batch.edge_index_dict)


def train():
    model.train()

    total_examples = total_loss = 0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        batch = batch.to(device)
        batch_size = batch['paper'].batch_size
        out = model(batch.x_dict, batch.edge_index_dict)['paper'][:batch_size]
        loss = F.cross_entropy(out, batch['paper'].y[:batch_size])
        loss.backward()
        optimizer.step()

        total_examples += batch_size
        total_loss += float(loss) * batch_size

    return total_loss / total_examples


@torch.no_grad()
def test(loader):
    model.eval()

    total_examples = total_correct = 0
    for batch in tqdm(loader):
        batch = batch.to(device)
        batch_size = batch['paper'].batch_size
        out = model(batch.x_dict, batch.edge_index_dict)['paper'][:batch_size]
        pred = out.argmax(dim=-1)

        total_examples += batch_size
        total_correct += int((pred == batch['paper'].y[:batch_size]).sum())

    return total_correct / total_examples


init_params()  # Initialize parameters.
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1, 21):
    loss = train()
    val_acc = test(val_loader)
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Val: {val_acc:.4f}')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 254/254 [01:13<00:00,  3.45it/s]


Epoch: 01, Loss: 2.4314, Val: 0.3972


 59%|█████▊    | 1441/2460 [06:31<04:00,  4.23it/s]